## ARC 380 / CEE 380 – Introduction to Robotics for Digital Fabrication
## Session 19 Workshop
Princeton University, Spring 2024

Professor: Arash Adel | Assistant-in-Instruction: Daniel Ruan

---

Open3D documentation: https://www.open3d.org/docs/release/index.html

In [2]:
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [4]:
# %pip install scipy
from scipy.cluster.vq import kmeans2

# 0. Review: Point Cloud Filtering

We will be reusing the point cloud from the previous workshop. Before segmenting, we want to run through the same filtering steps.

In [5]:
# Import workshop point cloud
pcd = o3d.io.read_point_cloud("example_pcd.ply")

print(f'Loaded point cloud has {len(pcd.points)} points.')

Loaded point cloud has 325101 points.


In [6]:
# Downsample the point cloud using voxel grid
voxel_size = 0.001   # Meters
down_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
print(f'Downsampled point cloud has {len(down_pcd.points)} points.')

Downsampled point cloud has 231940 points.


In [7]:
# Remove points beyond a certain z distance from the camera
max_distance = 0.5   # Meters

np_down_pcd = np.asarray(down_pcd.points)
within_dist_idx = np.where(np.abs(np_down_pcd[:, 2]) < max_distance)[0]
filtered_pcd = down_pcd.select_by_index(within_dist_idx)

print(f'Filtered point cloud has {len(filtered_pcd.points)} points.')

Filtered point cloud has 101980 points.


In [8]:
# Remove outlier points using statistical outlier removal
nb_neighbors = 20
std_ratio = 2
filtered_pcd, idx = filtered_pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)

print(f'Outlier removed point cloud has {len(filtered_pcd.points)} points.')

Outlier removed point cloud has 99188 points.


In [9]:
# Visualize filtered point cloud
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
rotation = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
coordinate_frame.rotate(rotation, center=(0, 0, 0))
o3d.visualization.draw_geometries([filtered_pcd, coordinate_frame])

---
# 1. Point cloud segmentation

## 1.1. Model-Based Methods

If we have prior information about the point cloud, then we can try and fit a model to it. For example, we know that a significant portion of our point cloud is a table, so we can fit a plane and its inliers should correspond with the surface.

In [10]:
# Use RANSAC to fit a plane and locate the table surface
plane_model, inliers = filtered_pcd.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)
[a, b, c, d] = plane_model
print(f'Plane model: {a}x + {b}y + {c}z + {d} = 0')

Plane model: -0.004938893277066706x + 0.11406548771480494y + 0.9934609563770392z + 0.3009804677684685 = 0


In [11]:
# Visualize the inliers of the plane
inlier_pcd = filtered_pcd.select_by_index(inliers)
inlier_pcd.paint_uniform_color([0, 1, 0])
print(f'Plane inliers point cloud has {len(inlier_pcd.points)} points.')

outlier_pcd = filtered_pcd.select_by_index(inliers, invert=True)
outlier_pcd.paint_uniform_color([1, 0, 0])
print(f'Plane outliers point cloud has {len(outlier_pcd.points)} points.')

o3d.visualization.draw_geometries([inlier_pcd, outlier_pcd, coordinate_frame])

Plane inliers point cloud has 97624 points.
Plane outliers point cloud has 1564 points.


Note, however, that the distance threshold can greatly impact our results. If we try using RANSAC again on the outlier point cloud to get one of the block surfaces, we need to utilize a smaller `distance_threshold` value for it to correctly segment the two surfaces.

In [12]:
# Use RANSAC again to try and get one of the block surfaces
plane_model_2, inliers_2 = outlier_pcd.segment_plane(distance_threshold=0.002, ransac_n=3, num_iterations=1000)
[a, b, c, d] = plane_model_2
print(f'Plane model: {a}x + {b}y + {c}z + {d} = 0')

# Visualize the inliers of the plane
inlier_pcd_2 = outlier_pcd.select_by_index(inliers_2)
inlier_pcd_2.paint_uniform_color([0, 1, 0])
print(f'Plane inliers point cloud has {len(inlier_pcd_2.points)} points.')

outlier_pcd_2 = outlier_pcd.select_by_index(inliers_2, invert=True)
outlier_pcd_2.paint_uniform_color([1, 0, 0])
print(f'Plane outliers point cloud has {len(outlier_pcd_2.points)} points.')

o3d.visualization.draw_geometries([inlier_pcd_2, outlier_pcd_2, coordinate_frame])

Plane model: -0.01250694030809919x + 0.10503517120711803y + 0.9943898577789403z + 0.24493000956221778 = 0
Plane inliers point cloud has 817 points.
Plane outliers point cloud has 747 points.


We can also potentially get undesirable results if, for example, we had multiple stacks of blocks that were the same height. RANSAC would fit a plane through each stack, and not properly segment them.

## 1.2. Unsupervised Learning Methods

### 1.2.1. K-means clustering

We previously used K-means clustering to segment 2D images. When we start getting into higher dimensional spaces, however, K-means clustering does not work as well. For example, we do not have significant variation along the z-axis (height), and so k-means will fail to segment the filtered point cloud.

In [13]:
# Use SciPy K-means clustering to segment the filtered point cloud
np_filtered_pcd = np.asarray(filtered_pcd.points)
k = 3
centroids, labels = kmeans2(np_filtered_pcd, k, iter=100, minit='points')
print(f'Centroids: \n{centroids}')

# Visualize the labeled point cloud
colors = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
colored_points = colors[labels]
colored_pcd = o3d.geometry.PointCloud()
colored_pcd.points = o3d.utility.Vector3dVector(np_filtered_pcd)
colored_pcd.colors = o3d.utility.Vector3dVector(colored_points)

# Create a sphere for each centroid
spheres = []
for centroid in centroids:
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.003)
    sphere.translate(centroid)
    spheres.append(sphere)

o3d.visualization.draw_geometries([colored_pcd, coordinate_frame] + spheres)


Centroids: 
[[ 0.00720219 -0.09317704 -0.29230056]
 [ 0.02533757  0.06009382 -0.30793097]
 [-0.15747076 -0.00608094 -0.30274659]]


Apply k-means clustering after segmenting out the table points (e.g., using RANSAC) will provide better results since the clusters are more clearly separated along all dimensions.

In [14]:
# Use SciPy K-means clustering to segment the outlier point cloud after the first RANSAC
np_outlier_pcd = np.asarray(outlier_pcd.points)
k = 2   # We know there are two block surfaces
centroids_2, labels_2 = kmeans2(np_outlier_pcd, k, iter=100, minit='points')
print(f'Centroids: \n{centroids_2}')

# Visualize the labeled point cloud
colors_2 = np.array([[1, 0, 0], [0, 1, 0]])
colored_points_2 = colors_2[labels_2]
colored_pcd_2 = o3d.geometry.PointCloud()
colored_pcd_2.points = o3d.utility.Vector3dVector(np_outlier_pcd)
colored_pcd_2.colors = o3d.utility.Vector3dVector(colored_points_2)

# Create a sphere for each centroid
spheres_2 = []
for centroid in centroids_2:
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.002)
    sphere.translate(centroid)
    spheres_2.append(sphere)

o3d.visualization.draw_geometries([colored_pcd_2, coordinate_frame] + spheres_2)

Centroids: 
[[-0.09196176 -0.00650727 -0.28806796]
 [-0.01521934  0.04918027 -0.25169808]]


### 1.2.2. DBSCAN

DBSCAN stands for Density-Based Spatial Clustering of Applications with Noise. Instead of weighing points equally like in k-means, DBSCAN looks at the density of the point neighborhoods. The more points that are clustered together, the higher its density, and the more likely it is to form a cluster.

Unlike k-means, DBSCAN does not require prior knowledge of the number of clusters in the point cloud. The clusters are determined by the epsilon value (`eps` parameter in the Open3D implementation), which is how far apart points can be from their neighborhood.

Source: 
- Ester, Martin, et al. "A density-based algorithm for discovering clusters in large spatial databases with noise." *kdd*. Vol. 96. No. 34. 1996.

In [15]:
# Basic segmentation of the filtered point cloud using DBSCAN clustering
labels = filtered_pcd.cluster_dbscan(eps=0.005, min_points=10, print_progress=True)
labels = np.asarray(labels)
print(f'Found {len(np.unique(labels))} clusters.')

Found 3 clusters.


In [28]:
# Compute the centroids and generate spheres for visualization
dbscan_centroids = []
dbscan_spheres = []
for label in np.unique(labels):
    cluster = filtered_pcd.select_by_index(np.where(labels == label)[0])
    #print(cluster)
    centroid = np.asarray(cluster.get_center())
    dbscan_centroids.append(centroid)

    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.002)
    sphere.translate(centroid)
    dbscan_spheres.append(sphere)

In [26]:
# Visualize the clusters in different colors
dbscan_clusters = []

for i in range(len(np.unique(labels))):
    cluster = filtered_pcd.select_by_index(np.where(labels == i)[0])
    cluster.paint_uniform_color(np.random.rand(3))
    dbscan_clusters.append(cluster)

o3d.visualization.draw_geometries(dbscan_clusters + dbscan_spheres + [coordinate_frame])

---

# 2. 3D Feature Extraction

Depending on the segmentation methods, we may already have some feature information about the point cloud.

For example, RANSAC plane segmentation will give us the surface normal vector, while k-means will give us the centroids.

We are primarily interested in the poses of each block (i.e., the centroid of the top surface where the vacuum gripper will attach), and the orientations.

## 2.1. Principal Component Analysis

Recall that for 2D images, we used moments to compute various measures about segmented regions. We can theoretically do the same for point clouds; however, it will not be as informative. This is because our point cloud only contains (some) points from the surface of the object, and not any interior points, and so analysis of the distribution of points (especially in higher dimensions) becomes less intuitive.

We can instead use Principal Component Analysis (PCA) to determine the principal axes of a cluster. PCA originates from statistics, and is used to identify correlations (i.e., variance) in a data set. We expect that the long axis of our block to match the direction of greatest spread (i.e., largest variance) of our point cloud cluster.

Imagine we fit an ellipsoid to the cluster. In 3D space, there will be 3 principal axes, which correspond to the orthogonal vectors along the ellipsoid's length, width, and height.

One method for performing PCA is using Singular Value Decomposition (SVD):

$$\mathbf{A} = \mathbf{U\Sigma V}^\top$$

Recall that we previously used SVD for calculating the "average" 3D rotation transformation for the Iterative Closest Point (ICP) algorithm. Here, we are only interested in the columns of $\mathbf{V}$, which are the right singular vectors, and correspond to the principal components of $\mathbf{A}$, the covariance matrix of our point cloud.

There are other methods for performing PCA, but SVD is widely used because it is efficient to compute.

To compute the covariance matrix $\mathbf{A}$, we first center the point cloud on the cluster centroid (mean) to remove bias.

In [29]:
# Potentially change the index here if the chosen cluster is the table surface (we can use size information to determine this, for example)
cluster = dbscan_clusters[1]

# Center the cluster points around the mean
points = np.asarray(cluster.points)
mean = np.mean(points, axis=0)
centered_points = points - mean

print(f'Mean: {mean}')
print(f'Cluster shape: {centered_points.shape}')

Mean: [-0.01521934  0.04918027 -0.25169808]
Cluster shape: (817, 3)


By convention, each row of our input matrix should be a variable that we are interested in computing the covariance with. In our case, we want to find the covariance along the x, y, and z axes, which are currently the columns of the point cloud (i.e., it currently has shape (N, 3)). As such, we need to transpose the point cloud first before computing the covariance matrix.

In [30]:
# Compute the covariance matrix of the cluster
cov = np.cov(centered_points.T)

print(f'Covariance matrix: \n{cov}')

Covariance matrix: 
[[ 1.63279010e-04 -1.86797076e-05  4.02673769e-06]
 [-1.86797076e-05  2.79837358e-05 -3.19080333e-06]
 [ 4.02673769e-06 -3.19080333e-06  5.04822075e-07]]


We can now proceed with SVD to compute the principal axes. Remember that SVD gives us the transpose $\mathbf{V}^\top$, so we need to transpose it back to get $\mathbf{V}$.

In [32]:
# Perform SVD on the covariance matrix
u, sigma, v_transpose = np.linalg.svd(cov)

v = v_transpose.T   # Transpose to get the right singular vectors

print(f'Singular values: {sigma}')
print(f'Right singular vectors (columns of V): \n{v}')

Singular values: [1.65928743e-04 2.57230027e-05 1.15822125e-07]
Right singular vectors (columns of V): 
[[-0.9905198  -0.13680362 -0.01246182]
 [ 0.13474824 -0.9852479   0.10549643]
 [-0.02671028  0.10281709  0.99434159]]


Note that the principal axes correspond with the singular values. The largest singular value corresponds to the first principal axis (the 'length' of the ellipsoid), and so on.

I.e., if the first singular value in $\mathbf{\Sigma}$ is the largest, then the first column in $\mathbf{V}$ is the largest principal axis, which in our case you can imagine as the 'length' of the fit ellipsoid. The smallest singular value will correspond to the 'height' of the ellipsoid, or the normal vector of the surface.

By default, NumPy's SVD function will sort the singular values in descending order.

We can visualize the principal axes by creating a coordinate frame centered at the cluster mean, and rotate it by $\mathbf{V}$ (remember that $\mathbf{V}$ is orthonormal, i.e., a rotation matrix).

In [ ]:
# Visualize the cluster with its principal axes
cluster_axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.05, origin=mean)
cluster_axes.rotate(v, center=mean)

o3d.visualization.draw_geometries([cluster, cluster_axes, coordinate_frame])

The cluster mean and principal axes give us enough information to construct the pickup frame for the block relative to the camera.

To transform it to the world frame, you can use the same techniques from Assignment 5 to localize the camera using the image, fiducial markers, and taught points, then apply that transformation to the pickup frame.

**Additional resources:**
- StatQuest (YouTube):
  - StatQuest: K-means clustering - https://www.youtube.com/watch?v=4b5d3muPQmA
  - Clustering with DBSCAN, Clearly Explained!!! - https://www.youtube.com/watch?v=RDZUdRSDOok
  - StatQuest: Principal Component Analysis (PCA), Step-by-Step - https://www.youtube.com/watch?v=FgakZw6K1QQ
- Gareth James, Daniela Witten, Trevor Hastie, Robert Tibshirani. (2013). An introduction to statistical learning : with applications in Python. New York: Springer.
  - https://www.statlearning.com/